## Get to grid testing script
Fletcher Newman

November 13th, 2024

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import files and convert columns to bool
sched = pd.read_excel('getto_format.xlsx')
staff = pd.read_excel('staff_getto.xlsx')
staff[['Gender', 'Lifegaurd', 'Ropes', 'Leadership', 'Boatie']] = staff[['Gender', 'Lifegaurd', 'Ropes', 'Leadership', 'Boatie']].astype('bool')
sched[['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']] = sched[['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']].astype(bool)
sched = sched.fillna('')


In [12]:
sched.query('Activity == "Lifegaurd"')

,Activity,Start,End,Require,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday
9,Lifegaurd,945,1045,"Lifegaurd, Workcrew",False,False,False,False,False,True
10,Lifegaurd,945,1045,"Lifegaurd, Workcrew",False,False,False,False,False,True
11,Lifegaurd,945,1030,"Lifegaurd, Workcrew",False,False,True,False,False,False
12,Lifegaurd,945,1030,"Lifegaurd, Workcrew",False,True,True,False,False,False
13,Lifegaurd,1030,1145,"Lifegaurd, Workcrew",False,True,True,False,False,False
14,Lifegaurd,1030,1145,"Lifegaurd, Workcrew",False,True,True,False,False,False
22,Lifegaurd,1345,1515,"Lifegaurd, Workcrew",False,True,True,True,True,True
23,Lifegaurd,1345,1515,"Lifegaurd, Workcrew",False,True,True,True,True,True
24,Lifegaurd,1500,1600,Lifegaurd,True,True,True,True,True,True
25,Lifegaurd,1500,1600,Lifegaurd,True,True,True,True,True,True


In [4]:
# Create output dataframe
grid = pd.DataFrame(sched['Activity'])

# Create time list
times = []
for i in range(len(sched['Start'])):
    # Handle start time string
    s1 = int(sched['Start'][i] // 100)
    if s1 > 12:
        s1 -= 12
    s1 = str(s1)
    s2 = int(sched['Start'][i] % 100)
    add = ''
    if s2 == 0:
        add = '0'
    s2 = str(s2) + add
    start = s1 + ':' + s2

    # handle end time string
    e1 = int(sched['End'][i] // 100)
    if e1 > 12:
        e1 -= 12
    e1 = str(e1)
    e2 = int(sched['End'][i] % 100)
    add = ''
    if e2 == 0:
        add = '0'
    e2 = str(e2) + add
    end = e1 + ':' + e2

    times.append(start + '-' + end)

grid['Time'] = times



In [5]:
grid

,Activity,Time
0,Sunrise Breakast Helper,7:00-8:00
1,Sunrise Breakast Helper,7:00-8:00
2,Turf Time,7:00-8:15
3,Turf Time,7:00-8:15
4,Coffee Shop,7:00-8:30
...,...,...
56,LG Spotter,9:00-10:00
57,Games,9:00-10:00
58,Games,9:00-10:00
59,TAPS,11:00-11:45


In [6]:
# Start assignment loop
week = 1
if week < 6:
    half = 1
else: 
    half = 2
    
numStaff = len(staff)
days = {0: 'Sunday', 1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}
freqLevel = 0

for day in days:
    currDay = days[day]
    dayList = []

    for i in range(len(sched)):
        # Check if activity is scheduled that day
        if sched[currDay][i]:
            index = np.random.randint(0, len(staff))    # Sets random index to start checking staff 
            repeat = 0
            found = False
            noneFound = False

            # Loop through staff to see who can do it
            while not found:
                repeat += 1

                # If search finds no one avaible
                if repeat > numStaff:
                    noneFound = True
                    found = True
                    continue

                # Check if correct half
                if staff['Half'][index] not in [half, 3]:
                    index = (index + 1) % numStaff
                    continue

                # Check if they have been scheduled recently (1 hour buffer between when they get off and when they can start again)
                if staff['prevDay'][index] == day:
                    if staff['prevTime'][index] + 1 > sched['Start'][i]:
                        index = (index + 1) % numStaff
                        continue

                # Check if they meet the requirements
                req = sched['Require'][i].split(',')
                if '' not in req:
                    metReq = True
                    for crit in req:
                        crit = crit.strip()
                        if not staff[crit][index]:
                            metReq = False
                            break
                    if not metReq:
                        index = (index + 1) % numStaff
                        continue

                # Passed all requirements, this is the one
                found = True

            # Check if returned empty
            if noneFound:
                dayList.append('NONE FOUND')

            # add staff to list and update their recent time attributes 
            else:
                dayList.append(staff['Camp_name'][index])
                staff.loc[index, 'prevDay'] = day
                staff.loc[index, 'prevTime'] = sched['End'][i]

        else:
            dayList.append('')

    grid[currDay] = dayList  





            


In [7]:
grid

,Activity,Time,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday
0,Sunrise Breakast Helper,7:00-8:00,,Unicorn,Claps,Juice,T-Wisk,Unicorn
1,Sunrise Breakast Helper,7:00-8:00,,Acro,Ups,Geni,Bow Wow,Padge
2,Turf Time,7:00-8:15,,Bow Wow,,,,Bolt
3,Turf Time,7:00-8:15,,Padge,,,,Pizza
4,Coffee Shop,7:00-8:30,,Gaggles,Gaggles,Gaggles,Gaggles,Gaggles
...,...,...,...,...,...,...,...,...
56,LG Spotter,9:00-10:00,Bolt,Ups,,,,
57,Games,9:00-10:00,,Nodder,Ups,,Hawma,
58,Games,9:00-10:00,Ups,Juice,Nono,Sea,Nodder,Goobie
59,TAPS,11:00-11:45,Geni,Geni,Geni,Geni,Geni,Geni


In [8]:
file_name = 'Grids/GetToGrid_Week' + str(week) + '.xlsx'
grid.to_excel(file_name)

In [15]:
staff['prevDay'] = np.zeros(len(staff['prevDay'])).astype(int)
staff['prevDay']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
Name: prevDay, dtype: int64